Langkah 1 Import Library

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
!pip uninstall transformers -y
!pip install transformers peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import pandas as pd
from transformers import  AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, IntervalStrategy
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import numpy as np
import os
os.environ["WANDB_DISABLED"] = "true"

Langkah 2 Memasukan dataset dan tahap pengolahan data

In [4]:
df = pd.read_csv("ada labelnya - Sheet1.csv")
df.rename(columns={'Data Komentar TIKTOK': 'text', 'Labeling': 'label'}, inplace=True)

print(f"Nilai kosong pada kolom text: {df['text'].isna().sum()}")
print(f"Nilai kosong pada kolom label: {df['label'].isna().sum()}")

df = df.dropna(subset=['text'])

print(f"DataFrame shape after dropping NA: {df.shape}")

label_map = {'netral': 0, 'positif': 1, 'negatif': 2}
df['label'] = df['label'].map(label_map)
df

Nilai kosong pada kolom text: 11
Nilai kosong pada kolom label: 0
DataFrame shape after dropping NA: (988, 2)


,text,label
0,Aku sebagai pemakai Hb noera bangga banget con...,1
1,mending noera apa lauskin?,0
2,tapi kok di aku ga ngaruh,2
3,"ikut gemeteran karena aku pake noera, ini cepe...",1
4,Yeayy bangga banget sebagai pemakai hb noera,1
...,...,...
993,Yeahhh ga sia2 dong beli HB dari,0
995,Coba mi renk Arta lolos apa enggak makasih,1
996,"tolong review beauty Rossa dong dok,mau beli m...",0
997,Semangat Doktif,0


Langkah 3 Inisialisasi model

In [5]:
model_name = "cahya/roberta-base-indonesian-522M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/926k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/468k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/507M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/507M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cahya/roberta-base-indonesian-522M and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Langkah 4 Tokenisasi Data

In [6]:
def tokenize_function(examples):
    texts = [str(text) if text is not None else "" for text in examples['text']]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=512)

dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/988 [00:00<?, ? examples/s]

Langkah 5 Split data

In [7]:
dataset = dataset.train_test_split(test_size=0.2)

train_dataset = dataset["train"]
test_dataset = dataset["test"]

Langkah 6 Load Model

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_map))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cahya/roberta-base-indonesian-522M and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Langkah 7 Set training config & trainer

In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy=IntervalStrategy.EPOCH,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Langkah 8 Memulai training model

In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.696200,0.561377
2,0.451300,0.564124
3,0.280800,0.783670


TrainOutput(global_step=297, training_loss=0.4671760480411928, metrics={'train_runtime': 246.8053, 'train_samples_per_second': 9.603, 'train_steps_per_second': 1.203, 'total_flos': 623578800015360.0, 'train_loss': 0.4671760480411928, 'epoch': 3.0})

Langkah 9 Evaluasi Model

In [11]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.7836697697639465, 'eval_runtime': 5.1403, 'eval_samples_per_second': 38.519, 'eval_steps_per_second': 4.864, 'epoch': 3.0}


Langkah 10 Simpan hasil sentimen ke file excel baru

In [ ]:
sentiment_results = pd.DataFrame()

predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argot(-1)

sentiment_results["text"] = test_dataset["text"]
sentiment_results["true_label"] = test_dataset["label"]
sentiment_results["predicted_label"] = predicted_labels

reverse_label_map = {v: k for k, v in label_map.items()}
sentiment_results['true_label_text'] = sentiment_results['true_label'].map(reverse_label_map)
sentiment_results['predicted_label_text'] = sentiment_results['predicted_label'].map(reverse_label_map)

# Calculate metrics
y_true = np.array(test_dataset["label"])
y_pred = predicted_labels

accuracy = accuracy_score(y_true, y_pred)
precision_macro = precision_score(y_true, y_pred, average='macro')
precision_weighted = precision_score(y_true, y_pred, average='weighted')
recall_macro = recall_score(y_true, y_pred, average='macro')
recall_weighted = recall_score(y_true, y_pred, average='weighted')
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

# Create classification report
report = classification_report(y_true, y_pred, target_names=list(reverse_label_map.values()), output_dict=True)

# Add metrics to results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (macro): {precision_macro:.4f}")
print(f"Precision (weighted): {precision_weighted:.4f}")
print(f"Recall (macro): {recall_macro:.4f}")
print(f"Recall (weighted): {recall_weighted:.4f}")
print(f"F1 Score (macro): {f1_macro:.4f}")
print(f"F1 Score (weighted): {f1_weighted:.4f}")

# Add metrics to DataFrame
for key, value in results.items():
  sentiment_results[key] = value

sentiment_results['accuracy'] = accuracy
sentiment_results['precision_macro'] = precision_macro
sentiment_results['precision_weighted'] = precision_weighted
sentiment_results['recall_macro'] = recall_macro
sentiment_results['recall_weighted'] = recall_weighted
sentiment_results['f1_macro'] = f1_macro
sentiment_results['f1_weighted'] = f1_weighted

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Save detailed metrics to a separate sheet
with pd.ExcelWriter("sentiment_analysis_results.xlsx") as writer:
  sentiment_results.to_excel(writer, sheet_name="Predictions", index=False)
  
  # Create metrics DataFrame
  metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision (macro)', 'Precision (weighted)', 
          'Recall (macro)', 'Recall (weighted)', 
          'F1 Score (macro)', 'F1 Score (weighted)'],
    'Value': [accuracy, precision_macro, precision_weighted, 
         recall_macro, recall_weighted, 
         f1_macro, f1_weighted]
  })
  metrics_df.to_excel(writer, sheet_name="Overall Metrics", index=False)
  
  # Class-specific metrics from classification report
  class_metrics = pd.DataFrame()
  for cls in report:
    if cls not in ['accuracy', 'macro avg', 'weighted avg']:
      class_metrics[cls] = pd.Series(report[cls])
  class_metrics.to_excel(writer, sheet_name="Class Metrics", index=True)

print("Sentiment analysis results saved to sentiment_analysis_results.xlsx")

Sentiment analysis results saved to sentiment_analysis_results.xlsx
